# RAG 성능 향상을 위한 문맥 압축 기법
이 노트북에서는 RAG 시스템의 효율을 높이기 위한 문맥 기반 압축(contextual compression) 기법을 구현합니다.    
검색된 텍스트 청크에서 핵심적인 부분만 남기고 불필요한 정보를 줄여, 응답 품질을 높이고 잡음을 최소화하는 것이 목표입니다.  

RAG에서 문서를 불러오면, 관련 정보와 무관한 내용이 섞인 청크가 함께 들어오는 경우가 많습니다. 문맥 압축은 다음과 같은 효과를 제공합니다.

- 관련 없는 문장이나 문단 제거
- 질의와 관련된 정보에만 집중
- 제한된 컨텍스트 창 안에 유용한 정보 밀도 최대화

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv


# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## 추출된 텍스트 청크 분할
텍스트를 추출한 뒤에는 검색 정확도를 높이기 위해 조금씩 겹치도록 나눠서 작은 단위로 분할(chunk)합니다.  

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 겹치는 n개의 문자 세그먼트로 청크합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 빈 리스트를 초기화합니다.
    
    # (n - overlap) 단계로 텍스트를 반복합니다.
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트를 청크 리스트에 추가합니다.
        chunks.append(text[i:i + n])

    return chunks  

## Simple Vector Store 구축
NumPy를 사용하여 간단한 Vecotr store 구축

In [ ]:
import numpy as np

class SimpleVectorStore:
    """
    NumPy를 사용하여 간단한 Vecotr store 구축
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목 추가

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5):
        """
        시맨틱 서치 수행

        Args:
        query_embedding (List[float]): 쿼리 임베딩 벡터.
        k (int): 반환할 결과의 수.

        Returns:
        List[Dict]: 텍스트와 메타데이터가 포함된 상위 k개 유사 항목.
        """
        if not self.vectors:
            return []
        
        # 쿼리 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도에 따라 정렬 (내림차순)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## 임베딩 생성

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

## 문서 처리 파이프라인
필요한 함수와 클래스를 정의했으니, 이제 문서 처리 파이프라인을 정의해 보겠습니다.

In [ ]:
def process_document(file_path, chunk_size=1000, chunk_overlap=200):
    """
    Process a document for RAG.

    Args:
    file_path (str): 파일 경로
    chunk_size (int): 각 청크의 크기(문자)
    chunk_overlap (int): 청크 간 중복 범위(문자)

    Returns:
    SimpleVectorStore: 문서 청크와 임베딩이 포함된 벡터 저장소
    """
    # PDF 파일에서 텍스트 추출
    # print("Extracting text from PDF...")
    # extracted_text = extract_text_from_pdf(pdf_path)

    # 텍스트 파일 로드
    extracted_text = load_text_file(file_path)
    
    # 추출된 텍스트를 청크로 분할
    print("Chunking text...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(chunks)} text chunks")
    
    # 각 텍스트 청크에 대한 임베딩 생성
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings(embedding_model, chunks, device=device, batch_size=1)
    
    # 벡터 저장소 생성
    store = SimpleVectorStore()
    
    # 각 청크와 해당 임베딩을 벡터 저장소에 추가
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": file_path}
        )
    
    print(f"Added {len(chunks)} chunks to the vector store")
    return store

## contextual compression 구현하기
이 부분은 이번 노트북의 핵심 부분입니다. LLM을 사용해서 검색된 콘텐츠를 필터링하고 압축(compression)합니다.


In [ ]:
def compress_chunk(chunk, query, compression_type="selective", model="gpt-4.1-mini"):
    """
    chunk를 압축하여 쿼리와 관련된 부분만 남기는 함수
    
    Args:
        chunk (str): 압축할 텍스트 청크
        query (str): 사용자 쿼리
        compression_type (str): 압축 유형 ("selective", "summary", or "extraction")
        model (str): 사용할 LLM 모델
        
    Returns:
        str: Compressed chunk
    """
    # 여러가지 압축 방법에 대한 시스템 프롬프트 정의
    if compression_type == "selective":
        system_prompt = """You are an expert at information filtering. 
        Your task is to analyze a document chunk and extract ONLY the sentences or paragraphs that are directly 
        relevant to the user's query. Remove all irrelevant content.

        Your output should:
        1. ONLY include text that helps answer the query
        2. Preserve the exact wording of relevant sentences (do not paraphrase)
        3. Maintain the original order of the text
        4. Include ALL relevant content, even if it seems redundant
        5. EXCLUDE any text that isn't relevant to the query

        Format your response as plain text with no additional comments."""
    elif compression_type == "summary":
        system_prompt = """You are an expert at summarization. 
        Your task is to create a concise summary of the provided chunk that focuses ONLY on 
        information relevant to the user's query.

        Your output should:
        1. Be brief but comprehensive regarding query-relevant information
        2. Focus exclusively on information related to the query
        3. Omit irrelevant details
        4. Be written in a neutral, factual tone

        Format your response as plain text with no additional comments."""
    else:  # extraction 
        system_prompt = """You are an expert at information extraction.
        Your task is to extract ONLY the exact sentences from the document chunk that contain information relevant 
        to answering the user's query.

        Your output should:
        1. Include ONLY direct quotes of relevant sentences from the original text
        2. Preserve the original wording (do not modify the text)
        3. Include ONLY sentences that directly relate to the query
        4. Separate extracted sentences with newlines
        5. Do not add any commentary or additional text

        Format your response as plain text with no additional comments."""

    # user prompt
    user_prompt = f"""
        Query: {query}

        Document Chunk:
        {chunk}

        Extract only the content relevant to answering this query.
    """
    
    # 응답 생성
    response = client_openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 압축된 chunk
    compressed_chunk = response.choices[0].message.content.strip()
    
    # 압축 비율 계산
    original_length = len(chunk)
    compressed_length = len(compressed_chunk)
    compression_ratio = (original_length - compressed_length) / original_length * 100
    
    return compressed_chunk, compression_ratio

## batch compressed 구현
효율성을 위해 가능하면 여러개의 chunk를 한 번에 압축합니다.

In [ ]:
def batch_compress_chunks(chunks, query, compression_type="selective", model="gpt-4.1-mini"):
    """
    여러 청크를 개별적으로 압축하는 함수
    
    Args:
        chunks (List[str]): 압축할 텍스트 청크 리스트
        query (str): 사용자 쿼리
        compression_type (str): 압축 유형 ("selective", "summary", or "extraction")
        model (str): 사용할 LLM 모델
        
    Returns:
        List[Tuple[str, float]]: 압축 비율이 포함된 압축된 청크 리스트
    """
    print(f"Compressing {len(chunks)} chunks...") # 한번에 압축할 청크 수 출력
    results = []  # 결과를 저장할 빈 리스트 초기화
    total_original_length = 0  # 청크의 총 원래 길이를 저장할 변수 초기화
    total_compressed_length = 0  # 청크의 총 압축된 길이를 저장할 변수 초기화
    
    # 각 청크에 대해 반복
    for i, chunk in enumerate(chunks):
        print(f"Compressing chunk {i+1}/{len(chunks)}...")  # 압축 진행 상태 출력
        # 청크를 압축하고 압축된 청크와 압축 비율을 반환
        compressed_chunk, compression_ratio = compress_chunk(chunk, query, compression_type, model)
        results.append((compressed_chunk, compression_ratio))  
        
        total_original_length += len(chunk)  
        total_compressed_length += len(compressed_chunk)  
    
    # 전체 압축 비율 계산
    overall_ratio = (total_original_length - total_compressed_length) / total_original_length * 100
    print(f"Overall compression ratio: {overall_ratio:.2f}%")  
    
    return results  

## Response Generation Function

In [ ]:
def generate_response(query, context, model_name="gpt-4.1-nano"):
    """
    쿼리와 컨텍스트를 기반으로 응답을 생성하는 함수
    
    Args:
        query (str): 사용자 쿼리
        context (str): 압축된 청크의 컨텍스트 텍스트
        model (str): 사용할 LLM 모델
        
    Returns:
        str: 생성된 응답
    """
    # 시스템 프롬프트
    system_prompt = """You are a helpful AI assistant. Answer the user's question based only on the provided context.
    If you cannot find the answer in the context, state that you don't have enough information."""
            
    # user 프롬프트
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        Please provide a comprehensive answer based only on the context above.
    """
    
    # 응답 생성
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    return response.choices[0].message.content

## Contextual Compression를 포함한 RAG Pipeline 

In [ ]:
def rag_with_compression(file_path, query, k=10, compression_type="selective", model_name="gpt-4.1-mini"):
    """
    RAG 파이프라인에 Contextual Compression 적용

    Args:
        file_path (str): 파일 경로
        query (str): 사용자 쿼리
        k (int): 초기 검색할 청크 수
        compression_type (str): 압축 유형
        model_name (str): 사용할 LLM 모델
        
    Returns:
        dict: 쿼리, 압축된 청크, 응답이 포함된 결과
    """
    print("\n=== RAG WITH CONTEXTUAL COMPRESSION ===")
    print(f"Query: {query}")
    print(f"Compression type: {compression_type}")
    
    # RAG에 사용할 문서에서 텍스트를 추출하고 청킹 후 임베딩 생성
    vector_store = process_document(file_path)
    
    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)
    
    # 쿼리 임베딩을 기반으로 가장 유사한 청크 검색
    print(f"Retrieving top {k} chunks...")
    results = vector_store.similarity_search(query_embedding, k=k)
    retrieved_chunks = [result["text"] for result in results]
    
    # 검색된 청크에 대해 압축 적용
    compressed_results = batch_compress_chunks(retrieved_chunks, query, compression_type, model_name)
    compressed_chunks = [result[0] for result in compressed_results]
    compression_ratios = [result[1] for result in compressed_results]
    
    # 빈 문자열로 압축된 청크 제거
    filtered_chunks = [(chunk, ratio) for chunk, ratio in zip(compressed_chunks, compression_ratios) if chunk.strip()]
    
    if not filtered_chunks:
        # 모든 청크가 빈 문자열로 압축된 경우 원본 청크 사용
        print("Warning: All chunks were compressed to empty strings. Using original chunks.")
        filtered_chunks = [(chunk, 0.0) for chunk in retrieved_chunks]
    else:
        compressed_chunks, compression_ratios = zip(*filtered_chunks)
    
    # 압축된 청크에서 컨텍스트 생성
    context = "\n\n---\n\n".join(compressed_chunks)
    
    # 압축된 청크를 기반으로 응답 생성
    print("Generating response based on compressed chunks...")
    response = generate_response(query, context, model_name)
    
    # 딕셔너리 형태로 저장
    result = {
        "query": query,
        "original_chunks": retrieved_chunks,
        "compressed_chunks": compressed_chunks,
        "compression_ratios": compression_ratios,
        "context_length_reduction": f"{sum(compression_ratios)/len(compression_ratios):.2f}%",
        "response": response
    }
    
    print("\n=== RESPONSE ===")
    print(response)
    
    return result

## Compression 여부에 따른 RAG 비교
일반 RAG와 Context Compression 기능이 추가된 버전을 비교하는 함수 생성

In [ ]:
def standard_rag(file_path, query, k=10, model_name="gpt-4.1-nano"):
    """
    Standard RAG without compression.
    
    Args:
        file_path (str): 파일 경로
        query (str): 사용자 쿼리
        k (int): 검색할 청크 수
        model_name (str): 사용할 LLM 모델
        
    Returns:
        dict: 쿼리, 청크, 응답이 포함된 결과
    """
    print("\n=== STANDARD RAG ===")
    print(f"Query: {query}")
    
    # RAG에 사용할 문서에서 텍스트를 추출하고 청킹 후 임베딩 생성
    vector_store = process_document(file_path)

    
    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)
    
    # 쿼리 임베딩을 기반으로 가장 유사한 청크 검색
    print(f"Retrieving top {k} chunks...")
    results = vector_store.similarity_search(query_embedding, k=k)
    retrieved_chunks = [result["text"] for result in results]
    
    # 검색된 청크에서 컨텍스트 생성
    context = "\n\n---\n\n".join(retrieved_chunks)
    
    # 검색된 청크를 기반으로 응답 생성
    print("Generating response...")
    response = generate_response(query, context, model_name)
    
    # 결과 딕셔너리 준비
    result = {
        "query": query,
        "chunks": retrieved_chunks,
        "response": response
    }
    
    print("\n=== RESPONSE ===")
    print(response)
    
    return result

## 평가

In [ ]:
def evaluate_responses(query, responses, reference_answer):
    """
    reference answer를 기준으로 여러 응답을 평가하는 함수
    
    Args:
        query (str): 사용자 쿼리
        responses (Dict[str, str]): 방법별 응답 딕셔너리
        reference_answer (str): 참조 답변
        
    Returns:
        str: 평가 텍스트
    """
    # 시스템 프롬프트 
    system_prompt = """You are an objective evaluator of RAG responses. Compare different responses to the same query
    and determine which is most accurate, comprehensive, and relevant to the query."""
    
    # 사용자 프롬프트 정의
    user_prompt = f"""
    Query: {query}

    Reference Answer: {reference_answer}

    """
    
    for method, response in responses.items():
        user_prompt += f"\n{method.capitalize()} Response:\n{response}\n"
    
    # 평가 기준을 사용자 프롬프트에 추가
    user_prompt += """
    Please evaluate these responses based on:
    1. Factual accuracy compared to the reference
    2. Comprehensiveness - how completely they answer the query
    3. Conciseness - whether they avoid irrelevant information
    4. Overall quality

    Rank the responses from best to worst with detailed explanations.
    """

    #### 여기까지 사용자 프롬프트 정의 과정 ####
    
    # 평가 응답 생성
    evaluation_response = client_openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    return evaluation_response.choices[0].message.content

In [ ]:
def evaluate_compression(file_path, query, reference_answer=None, compression_types=["selective", "summary", "extraction"]):
    """
    Contextual Compression 기술을 Standard RAG와 비교하는 함수
    
    Args:
        file_path (str): 파일 경로
        query (str): 사용자 쿼리
        reference_answer (str): 선택적 참조 답변
        compression_types (List[str]): 평가할 압축 유형
        
    Returns:
        dict: Evaluation results
    """
    print("\n=== EVALUATING CONTEXTUAL COMPRESSION ===")
    print(f"Query: {query}")
    
    # 압축 없이 표준 RAG 실행
    standard_result = standard_rag(file_path, query)
    
    compression_results = {}
    
    # 각각의 Contextual Compression 기술에 대해 RAG 실행
    for comp_type in compression_types:
        print(f"\nTesting {comp_type} compression...")
        compression_results[comp_type] = rag_with_compression(file_path, query, compression_type=comp_type)
    
    # 평가를 위한 응답 생성
    responses = {
        "standard": standard_result["response"]
    }
    for comp_type in compression_types:
        responses[comp_type] = compression_results[comp_type]["response"]
    
    # 정답(참조) 답변이 제공된 경우 응답 평가
    if reference_answer:
        evaluation = evaluate_responses(query, responses, reference_answer)
        print("\n=== EVALUATION RESULTS ===")
        print(evaluation)
    else:
        evaluation = "No reference answer provided for evaluation."
    
    # 각각의 Contextual Compression 기술을 이용하여 생성한 답변에 대한 점수 계산
    metrics = {}
    for comp_type in compression_types:
        metrics[comp_type] = {
            "avg_compression_ratio": f"{sum(compression_results[comp_type]['compression_ratios'])/len(compression_results[comp_type]['compression_ratios']):.2f}%",
            "total_context_length": len("\n\n".join(compression_results[comp_type]['compressed_chunks'])),
            "original_context_length": len("\n\n".join(standard_result['chunks']))
        }
    
    # 평가 결과, 응답, 점수 반환
    return {
        "query": query,
        "responses": responses,
        "evaluation": evaluation,
        "metrics": metrics,
        "standard_result": standard_result,
        "compression_results": compression_results
    }

## Running Our Complete System (Custom Query)

In [ ]:
import pandas as pd

# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')

# 평가 데이터에서 첫 번째 쿼리 추출
query = df['query'][0]

# 참고(정답) 답변
reference_answer = df['generation_gt'][0]

# 파일 경로
file_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"
file_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

# Run evaluation with different compression techniques  
# Compression types:  
# - "selective": Retains key details while omitting less relevant parts  
# - "summary": Provides a concise version of the information  
# - "extraction": Extracts relevant sentences verbatim from the document  
results = evaluate_compression(  
    file_path=file_path,  
    query=query,  
    reference_answer=reference_answer,  
    compression_types=["selective", "summary", "extraction"]  
)